In [1]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

In [2]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 52.472883,
	"longitude": 13.405122,
	"start_date": "1990-01-01",
	"end_date": "2024-12-31",
	"hourly": ["temperature_2m", "relative_humidity_2m", "precipitation", "soil_temperature_0_to_7cm", "soil_temperature_7_to_28cm", "soil_temperature_28_to_100cm", "soil_temperature_100_to_255cm", "soil_moisture_0_to_7cm", "soil_moisture_7_to_28cm", "soil_moisture_28_to_100cm", "soil_moisture_100_to_255cm"],
    "timezone": "Europe/Berlin"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(2).ValuesAsNumpy()
hourly_soil_temperature_0_to_7cm = hourly.Variables(3).ValuesAsNumpy()
hourly_soil_temperature_7_to_28cm = hourly.Variables(4).ValuesAsNumpy()
hourly_soil_temperature_28_to_100cm = hourly.Variables(5).ValuesAsNumpy()
hourly_soil_temperature_100_to_255cm = hourly.Variables(6).ValuesAsNumpy()
hourly_soil_moisture_0_to_7cm = hourly.Variables(7).ValuesAsNumpy()
hourly_soil_moisture_7_to_28cm = hourly.Variables(8).ValuesAsNumpy()
hourly_soil_moisture_28_to_100cm = hourly.Variables(9).ValuesAsNumpy()
hourly_soil_moisture_100_to_255cm = hourly.Variables(10).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["precipitation"] = hourly_precipitation
hourly_data["soil_temperature_0_to_7cm"] = hourly_soil_temperature_0_to_7cm
hourly_data["soil_temperature_7_to_28cm"] = hourly_soil_temperature_7_to_28cm
hourly_data["soil_temperature_28_to_100cm"] = hourly_soil_temperature_28_to_100cm
hourly_data["soil_temperature_100_to_255cm"] = hourly_soil_temperature_100_to_255cm
hourly_data["soil_moisture_0_to_7cm"] = hourly_soil_moisture_0_to_7cm
hourly_data["soil_moisture_7_to_28cm"] = hourly_soil_moisture_7_to_28cm
hourly_data["soil_moisture_28_to_100cm"] = hourly_soil_moisture_28_to_100cm
hourly_data["soil_moisture_100_to_255cm"] = hourly_soil_moisture_100_to_255cm

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)

Coordinates 52.478031158447266°N 13.38290023803711°E
Elevation 48.0 m asl
Timezone b'Europe/Berlin' b'GMT+2'
Timezone difference to GMT+0 7200 s
                            date  temperature_2m  relative_humidity_2m  \
0      1989-12-31 22:00:00+00:00          -1.326             94.983139   
1      1989-12-31 23:00:00+00:00          -1.176             94.291893   
2      1990-01-01 00:00:00+00:00          -1.376             94.282692   
3      1990-01-01 01:00:00+00:00          -1.376             91.534355   
4      1990-01-01 02:00:00+00:00          -1.526             87.857422   
...                          ...             ...                   ...   
306811 2024-12-31 17:00:00+00:00           1.011             90.369087   
306812 2024-12-31 18:00:00+00:00           0.961             88.411232   
306813 2024-12-31 19:00:00+00:00           1.311             85.901741   
306814 2024-12-31 20:00:00+00:00           1.111             85.254410   
306815 2024-12-31 21:00:00+00:00         

In [3]:
hourly_dataframe.head()

,date,temperature_2m,relative_humidity_2m,precipitation,soil_temperature_0_to_7cm,soil_temperature_7_to_28cm,soil_temperature_28_to_100cm,soil_temperature_100_to_255cm,soil_moisture_0_to_7cm,soil_moisture_7_to_28cm,soil_moisture_28_to_100cm,soil_moisture_100_to_255cm
0,1989-12-31 22:00:00+00:00,-1.326,94.983139,0.0,-0.426,0.724,4.324,9.024,0.35,0.353,0.305,0.292
1,1989-12-31 23:00:00+00:00,-1.176,94.291893,0.0,-0.376,0.674,4.274,9.024,0.35,0.352,0.305,0.292
2,1990-01-01 00:00:00+00:00,-1.376,94.282692,0.0,-0.426,0.674,4.274,9.024,0.35,0.352,0.305,0.292
3,1990-01-01 01:00:00+00:00,-1.376,91.534355,0.0,-0.426,0.674,4.274,9.024,0.35,0.352,0.305,0.292
4,1990-01-01 02:00:00+00:00,-1.526,87.857422,0.0,-0.426,0.674,4.274,9.024,0.35,0.352,0.305,0.292


In [4]:
hourly_dataframe.to_csv("thf-climatedata-1990-2024.csv", index=False)